In [21]:
!pip install playwright pandas tqdm nest_asyncio
!playwright install --with-deps chromium

Installing dependencies...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building depe

In [22]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright, Page
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio


In [23]:
df = pd.read_csv("/content/job_part1.csv")

In [24]:
# --- Helper Functions to Scrape Specific Details ---
async def get_nganh_nghe_from_page(page: Page) -> str:
    """
    Trích xuất thông tin Ngành nghề từ trang chi tiết công việc.
    """
    # print("INFO: Đang tìm và trích xuất mục: 'Ngành nghề'") # Optional: for verbose logging
    try:
        # Attempts to find an <li> element that contains a <strong> with "Ngành nghề"
        # and then extracts text from the <p> > <a> element within that <li>.
        label_element = await page.query_selector("xpath=//li[.//strong[contains(normalize-space(.), 'Ngành nghề')]]")
        if label_element:
            value_element = await label_element.query_selector("p > a")
            if value_element:
                text = (await value_element.inner_text()).strip()
                if text:
                    return text

        # Fallback: Try finding a <strong> containing "Ngành nghề" and then its sibling <p> > <a>
        strong_nganh_nghe = await page.query_selector("xpath=//strong[contains(normalize-space(.), 'Ngành nghề')]")
        if strong_nganh_nghe:
            # Try to find the parent li first, then the p > a structure, which is common
            parent_li = await strong_nganh_nghe.query_selector("xpath=ancestor::li")
            if parent_li:
                 p_tag = await parent_li.query_selector("p")
                 if p_tag:
                    a_tag = await p_tag.query_selector("a")
                    if a_tag:
                        text = (await a_tag.inner_text()).strip()
                        if text: return text
            else: # If not in an li, try direct sibling p > a
                p_sibling = await strong_nganh_nghe.query_selector("xpath=./following-sibling::p[1]")
                if p_sibling:
                    a_tag = await p_sibling.query_selector("a")
                    if a_tag:
                        text = (await a_tag.inner_text()).strip()
                        if text: return text
        # print("LƯU Ý: Không tìm thấy 'Ngành nghề' với các selector đã thử.") # Optional
    except Exception as e:
        print(f"LỖI khi trích xuất Ngành nghề cho {page.url}: {e}")
    return "N/A"

async def get_hinh_thuc_from_page(page: Page) -> str:
    """
    Trích xuất thông tin Hình thức từ trang chi tiết công việc.
    """
    # print("INFO: Đang tìm và trích xuất mục: 'Hình thức'") # Optional
    try:
        label_element = await page.query_selector("xpath=//li[.//strong[contains(normalize-space(.), 'Hình thức')]]")
        if label_element:
            value_element = await label_element.query_selector("p")
            if value_element:
                text = (await value_element.inner_text()).strip()
                if text:
                    return text

        strong_hinh_thuc = await page.query_selector("xpath=//strong[contains(normalize-space(.), 'Hình thức')]")
        if strong_hinh_thuc:
            parent_li = await strong_hinh_thuc.query_selector("xpath=ancestor::li")
            if parent_li:
                p_tag = await parent_li.query_selector("p")
                if p_tag:
                    text = (await p_tag.inner_text()).strip()
                    if text: return text
            else: # If not in an li, try direct sibling p
                p_sibling = await strong_hinh_thuc.query_selector("xpath=./following-sibling::p[1]")
                if p_sibling:
                    text = (await p_sibling.inner_text()).strip()
                    if text: return text
        # print("LƯU Ý: Không tìm thấy 'Hình thức' với các selector đã thử.") # Optional
    except Exception as e:
        print(f"LỖI khi trích xuất Hình thức cho {page.url}: {e}")
    return "N/A"

async def scrape_job_details(job_urls: list) -> pd.DataFrame:
    """
    Scrapes 'Ngành nghề' and 'Hình thức' for a list of job URLs.
    """
    results = []
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True) # Set to False to see browser for debugging
        context = await browser.new_context(user_agent=user_agent, java_script_enabled=True)
        page = await context.new_page()

        for url in tqdm_asyncio(job_urls, desc="Scraping job details"):
            if not isinstance(url, str) or not url.startswith("http"):
                print(f"URL không hợp lệ, bỏ qua: {url}")
                results.append({"URL": url, "NganhNghe": "Invalid URL", "HinhThuc": "Invalid URL"})
                continue

            nganh_nghe = "N/A"
            hinh_thuc = "N/A"

            try:
                await page.goto(url, wait_until="networkidle", timeout=60000) # networkidle can be more robust
                # await page.wait_for_timeout(1000) # Optional small delay if elements load slowly

                nganh_nghe = await get_nganh_nghe_from_page(page)
                hinh_thuc = await get_hinh_thuc_from_page(page)

                print(f"URL: {url} -> Ngành nghề: {nganh_nghe}, Hình thức: {hinh_thuc}")

            except Exception as e:
                print(f"Lỗi nghiêm trọng khi xử lý URL {url}: {e}")
                nganh_nghe = f"Lỗi: {str(e)[:50]}" # Store part of the error
                hinh_thuc = f"Lỗi: {str(e)[:50]}"

            results.append({"URL": url, "NganhNghe": nganh_nghe, "HinhThuc": hinh_thuc})
            await asyncio.sleep(1) # Be polite to the server

        await browser.close()

    return pd.DataFrame(results)

In [25]:
# --- Main Execution ---
async def main():
    # !!! REPLACE WITH YOUR LIST OF URLs !!!
    # Example URLs (use actual careerviet.vn job detail URLs):
    list_of_job_urls = df["URL"][:5].to_list()

    if not list_of_job_urls:
        print("Danh sách URL trống. Vui lòng cung cấp URL để cào dữ liệu.")
        return

    print(f"Bắt đầu cào dữ liệu cho {len(list_of_job_urls)} URLs...")
    detailed_df = await scrape_job_details(list_of_job_urls)

    print("\n--- Kết quả cào dữ liệu ---")
    if not detailed_df.empty:
        from google.colab import data_table # For better table display in Colab
        display(data_table.DataTable(detailed_df, include_index=False, num_rows_per_page=10))

        # Save to Excel
        output_filename = "careerviet_nganh_nghe_hinh_thuc.xlsx"
        detailed_df.to_excel(output_filename, index=False, engine='openpyxl')
        print(f"\nDữ liệu đã được lưu vào file: {output_filename}")
    else:
        print("Không có dữ liệu nào được cào.")

In [27]:
await main()

Bắt đầu cào dữ liệu cho 5 URLs...


Scraping job details:   0%|          | 0/5 [00:00<?, ?it/s]

URL: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-ho-tro-ky-thuat-o-to-thuong-mai.35C4A0D4.html -> Ngành nghề: Cơ khí / Ô tô / Tự động hóa, Hình thức: Nhân viên chính thức


Scraping job details:  20%|██        | 1/5 [00:16<01:04, 16.14s/it]

URL: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-phu-tung.35C4A0D3.html -> Ngành nghề: Tiếp thị / Marketing, Hình thức: Nhân viên chính thức


...

URL: https://careerviet.vn/vi/tim-viec-lam/phien-dich-vien-tieng-trung-quoc.35C4A0C1.html -> Ngành nghề: Hành chính / Thư ký, Hình thức: Nhân viên chính thức


Scraping job details: 100%|██████████| 5/5 [00:44<00:00,  8.87s/it]



--- Kết quả cào dữ liệu ---


,URL,NganhNghe,HinhThuc
0,https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-ho-tro-ky-thuat-o-to-thuong-mai.35C4A0D4.html,Cơ khí / Ô tô / Tự động hóa,Nhân viên chính thức
1,https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-phu-tung.35C4A0D3.html,Tiếp thị / Marketing,Nhân viên chính thức
2,https://careerviet.vn/vi/tim-viec-lam/pho-truong-bo-phan-phap-ly-dang-kiem-xe-cbu-ckd.35C4A0D1.html,Luật / Pháp lý,Nhân viên chính thức
3,https://careerviet.vn/vi/tim-viec-lam/tai-xe-lai-xe-cho-giam-doc.35C4A0C8.html,Vận chuyển / Giao nhận / Kho vận,Nhân viên chính thức
4,https://careerviet.vn/vi/tim-viec-lam/phien-dich-vien-tieng-trung-quoc.35C4A0C1.html,Hành chính / Thư ký,Nhân viên chính thức



Dữ liệu đã được lưu vào file: careerviet_nganh_nghe_hinh_thuc.xlsx
